In [3]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
import random
from citipy import citipy
from datetime import datetime 
from config1 import weather_api_key
import requests
import time
from scipy.stats import linregress
import gmaps
from config1 import g_key

In [4]:
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size = 2000)
lat_lngs = zip(lats,lngs)
coords = list(lat_lngs)

In [ ]:
# Retreiving nearest cities 

cities = []

for coordinate in coords:
    city = citipy.nearest_city(coordinate[0],coordinate[1]).city_name
    country = citipy.nearest_city(coordinate[0],coordinate[1]).country_code
    
    if city not in cities:
        cities.append(city)
len(cities)

In [2]:
# Create an empty list to hold the weather data
city_data = []

# Print the beginning of the logging
print("Beginning Data Retrieval       ")
print("-------------------------------")

# Create counters
record_count = 1
set_count = 1

url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

# Loop through all the cities in our list
for i, city in enumerate(cities):
    
    # Group cities in sets of 50 for logging purposes
    if (i%50 == 0 and i > 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city
    city_url = url + "&q=" + city.replace(" ","+")
    
    # Log the URL, record, and set numbers and city
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count
    record_count += 1

# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        #city_weather_description = city_weather["weather"][2]["description"]
        city_weather_description = city_weather["weather"][0]["description"]
        
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                         "Current Description": city_weather_description})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval       
-------------------------------


NameError: name 'weather_api_key' is not defined

In [15]:
print(city_data)

[{'City': 'Pevek', 'Lat': 69.7008, 'Lng': 170.3133, 'Max Temp': -15.93, 'Humidity': 98, 'Cloudiness': 94, 'Wind Speed': 7.81, 'Country': 'RU', 'Current Description': 'overcast clouds'}, {'City': 'Hervey Bay', 'Lat': -25.2986, 'Lng': 152.8535, 'Max Temp': 73, 'Humidity': 97, 'Cloudiness': 100, 'Wind Speed': 5.99, 'Country': 'AU', 'Current Description': 'overcast clouds'}, {'City': 'Touros', 'Lat': -5.1989, 'Lng': -35.4608, 'Max Temp': 86, 'Humidity': 66, 'Cloudiness': 40, 'Wind Speed': 14.97, 'Country': 'BR', 'Current Description': 'scattered clouds'}, {'City': 'Busselton', 'Lat': -33.65, 'Lng': 115.3333, 'Max Temp': 64, 'Humidity': 89, 'Cloudiness': 0, 'Wind Speed': 1.99, 'Country': 'AU', 'Current Description': 'clear sky'}, {'City': 'Sitka', 'Lat': 57.0531, 'Lng': -135.33, 'Max Temp': 35.6, 'Humidity': 93, 'Cloudiness': 90, 'Wind Speed': 8.05, 'Country': 'US', 'Current Description': 'light rain'}, {'City': 'Sao Filipe', 'Lat': 14.8961, 'Lng': -24.4956, 'Max Temp': 77.29, 'Humidity': 6

In [16]:
print(len(city_data))

661


In [17]:
city_data_df = pd.DataFrame(city_data)
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,Pevek,69.7008,170.3133,-15.93,98,94,7.81,RU,overcast clouds
1,Hervey Bay,-25.2986,152.8535,73.00,97,100,5.99,AU,overcast clouds
2,Touros,-5.1989,-35.4608,86.00,66,40,14.97,BR,scattered clouds
3,Busselton,-33.6500,115.3333,64.00,89,0,1.99,AU,clear sky
4,Sitka,57.0531,-135.3300,35.60,93,90,8.05,US,light rain


In [18]:
new_col_order = ["City", "Country", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Current Description"]
city_data_df = city_data_df[new_col_order]
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Pevek,RU,69.7008,170.3133,-15.93,98,94,7.81,overcast clouds
1,Hervey Bay,AU,-25.2986,152.8535,73.00,97,100,5.99,overcast clouds
2,Touros,BR,-5.1989,-35.4608,86.00,66,40,14.97,scattered clouds
3,Busselton,AU,-33.6500,115.3333,64.00,89,0,1.99,clear sky
4,Sitka,US,57.0531,-135.3300,35.60,93,90,8.05,light rain


In [19]:
# Create the output file (CSV).
output_data_file = "WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")